In [37]:
from PIL import Image, ImageDraw, ImageFont, ImageFilter


class ListSlideCreator:
    def __init__(self,
                 background_image_path="cyan cyan.png",
                 logo_path="logo_howai.png"):

        """
        Initialize the slide creator with background and logo images
        
        :param background_image_path: Path to the background image
        :param logo_path: Path to the logo image
        """
        self.im = Image.open(background_image_path)
        self.width = self.im.width
        self.height = self.im.height
        
        # Load logo
        self.logo = Image.open(logo_path).convert("RGBA")
        
        # Fonts
        self.font_header = "arialbd.ttf"
        self.fontsize_header = 300
        
        self.font_items = "arial.ttf"
        self.fontsize_items = 200

    def _create_gradient_mask(self, width, height):
        """
        Create a horizontal linear gradient image
        
        :param width: Width of the gradient
        :param height: Height of the gradient
        :return: Gradient image
        """
        # Create a new image for the gradient
        gradient = Image.new('L', (width, height), color=0)
        draw = ImageDraw.Draw(gradient)
        
        # Create gradient from left (white) to right (teal)
        for x in range(width):
            # Calculate gradient percentage
            gradient_percent = x / width
            
            # Interpolate between colors
            r = int(255 * (1 - gradient_percent) + 83 * gradient_percent)
            g = int(255 * (1 - gradient_percent) + 234 * gradient_percent)
            b = int(255 * (1 - gradient_percent) + 205 * gradient_percent)
            
            # Draw vertical line with interpolated color
            draw.line([(x, 0), (x, height)], 
                      fill=int(gradient_percent * 255))
        
        return gradient

    def _calculate_text_position(self, text, font):
        """
        Calculate the position of the text to be centered
        
        :param text: Text to be positioned
        :param font: ImageFont object
        :return: Tuple of (x, y) coordinates
        """
        # Create a temporary drawing context
        draw = ImageDraw.Draw(Image.new("RGB", (1, 1)))
        
        # Get text bounding box
        text_bbox = draw.textbbox((0, 0), text, font=font)
        text_width = text_bbox[2] - text_bbox[0]
        text_height = text_bbox[3] - text_bbox[1]
        
        # Center the text
        text_x = (self.width - text_width) // 2
        text_y = self.height // 10  # Positioning at top of slide
        
        return text_x, text_y, text_width, text_height

    def create_slide(self, header_text, list_items):
        """
        Create a slide with header and list items
        
        :param header_text: Text for the header
        :param list_items: List of items to display
        :return: Processed slide image
        """
        # Apply blur to background
        self.im = self.im.filter(ImageFilter.BLUR)
        
        # Prepare header font
        header_font = ImageFont.truetype(self.font_header, self.fontsize_header)
        
        # Calculate text position and bounding box
        text_x, text_y, text_width, text_height = self._calculate_text_position(header_text, header_font)
        
        # Create text layer
        text_layer = Image.new("RGBA", (text_width, text_height), (0, 0, 0, 0))
        text_draw = ImageDraw.Draw(text_layer)
        
        # Draw text in black
        text_draw.text((0, 0), header_text, font=header_font, fill=(0, 0, 0, 255))
        
        # Create gradient mask scaled to text size
        gradient_mask = self._create_gradient_mask(text_width, text_height)
        
        # Apply gradient to text
        gradient_text = Image.new("RGBA", (text_width, text_height), (0, 0, 0, 0))
        gradient_text.paste(text_layer, (0, 0), gradient_mask)
        
        # Paste gradient text onto the slide
        self.im.paste(gradient_text, (text_x, text_y), gradient_text)
        
        # Prepare list font
        list_font = ImageFont.truetype(self.font_items, self.fontsize_items)
        
        # Draw list items
        draw = ImageDraw.Draw(self.im)
        for i, item in enumerate(list_items):
            # Calculate vertical position for each item
            item_y = text_y + text_height + (i + 1) * list_font.getbbox(item)[1]
            
            # Draw list item
            draw.text((text_x, item_y), 
                      f"• {item}", 
                      font=list_font, 
                      fill=(17, 18, 43, 255))  # Dark blue color
        
        # Add logo to bottom left
        self.logo = self.logo.resize((int(self.logo.width * 0.5),
                                      int(self.logo.height * 0.5)))
        self.im.paste(self.logo, 
                      (0, self.im.height - self.logo.height), 
                      self.logo)
        
        return self.im

In [39]:
from PIL import ImageOps

class ListSlideCreator:
    def __init__(self, 
                 background_image_path="brain_shine_gradient_canvas.png", 
                 logo_path="logo_howai.png"):
        self.background_image_path = background_image_path
        self.logo_path = logo_path
        self.font_items_path = "arial.ttf"
        self.fontsize_items = 200
        self.font_header_path = "arialbd.ttf"
        self.fontsize_header = 300

    def create_gradient_text_image(self, text, font, start_color, end_color, width, height):
        # Create a base image for the gradient text
        gradient_image = Image.new("RGB", (width, height), start_color)
        draw = ImageDraw.Draw(gradient_image)
        
        # Create a mask with a horizontal gradient from black to white
        gradient_mask = Image.new("L", (width, height))
        for x in range(width):
            gradient_value = int(255 * (x / width))
            ImageDraw.Draw(gradient_mask).line([(x, 0), (x, height)], fill=gradient_value)

        # Blend start and end colors based on the gradient mask
        gradient_text = Image.composite(gradient_image, Image.new("RGB", (width, height), end_color), gradient_mask)

        # Draw the text on a transparent image
        text_image = Image.new("RGBA", (width, height), (255, 255, 255, 0))
        draw_text = ImageDraw.Draw(text_image)
        draw_text.text((0, 0), text, font=font, fill=(255, 255, 255, 255))
        
        # Ensure gradient_text is RGBA and combine with text image
        gradient_text = gradient_text.convert("RGBA")
        gradient_text = Image.alpha_composite(gradient_text, text_image)
        
        return gradient_text

    def create_slide(self, header_text, list_items):
        # Load background image
        background = Image.open(self.background_image_path).convert("RGBA")
        
        # Load the logo
        logo = Image.open(self.logo_path).convert("RGBA")

        # Define fonts
        font_header = ImageFont.truetype(self.font_header_path, self.fontsize_header)
        font_items = ImageFont.truetype(self.font_items_path, self.fontsize_items)
        
        # Create header text with gradient
        bbox = font_header.getbbox(header_text)
        header_width = bbox[2] - bbox[0]
        header_height = bbox[3] - bbox[1]
        gradient_header = self.create_gradient_text_image(header_text, font_header, 
                                                          (255, 255, 255), (83, 234, 205), 
                                                          header_width, header_height)
        
        # Position header on background
        header_x = (background.width - header_width) // 2
        header_y = 100  # Arbitrary top padding
        background.paste(gradient_header, (header_x, header_y), gradient_header)

        # Position logo at the bottom right
        logo.thumbnail((200, 200))  # Adjust size
        background.paste(logo, (background.width - logo.width - 20, background.height - logo.height - 20), logo)

        # Draw list items
        draw = ImageDraw.Draw(background)
        y_position = header_y + header_height + 100  # Space below header

        for item in list_items:
            draw.text((header_x, y_position), item, font=font_items, fill=(255, 255, 255))
            y_position += font_items.getbbox(item)[1] + 50  # Line spacing

        return background

# Adjustments have been made to ensure that the gradient text is applied correctly.


In [43]:
class ListSlideCreator:
    def __init__(self, 
                 background_image_path="brain_shine_gradient_canvas.png", 
                 logo_path="logo_howai.png"):
        self.background_image_path = background_image_path
        self.logo_path = logo_path
        self.font_items_path = "arial.ttf"
        self.fontsize_items = 200
        self.font_header_path = "arialbd.ttf"
        self.fontsize_header = 300

    def create_gradient_text_image(self, text, font, start_color, end_color, width, height):
        # Create a gradient background
        gradient_image = Image.new("RGB", (width, height), start_color)
        draw = ImageDraw.Draw(gradient_image)

        # Create a horizontal gradient mask
        gradient_mask = Image.new("L", (width, height))
        for x in range(width):
            gradient_value = int(255 * (x / width))
            ImageDraw.Draw(gradient_mask).line([(x, 0), (x, height)], fill=gradient_value)

        # Blend start and end colors across the width
        gradient_text = Image.composite(gradient_image, Image.new("RGB", (width, height), end_color), gradient_mask)

        # Draw the text mask
        text_mask = Image.new("L", (width, height), 0)
        draw_text_mask = ImageDraw.Draw(text_mask)
        draw_text_mask.text((0, 0), text, font=font, fill=255)
        
        # Apply the text mask to the gradient, so the text itself has the gradient color
        gradient_text_with_mask = Image.new("RGBA", (width, height), (255, 255, 255, 0))
        gradient_text_with_mask.paste(gradient_text, (0, 0), text_mask)

        return gradient_text_with_mask

    def create_slide(self, header_text, list_items):
        # Load background image
        background = Image.open(self.background_image_path).convert("RGBA")
        
        # Load the logo
        logo = Image.open(self.logo_path).convert("RGBA")

        # Define fonts
        font_header = ImageFont.truetype(self.font_header_path, self.fontsize_header)
        font_items = ImageFont.truetype(self.font_items_path, self.fontsize_items)
        
        # Create header text with gradient
        bbox = font_header.getbbox(header_text)
        header_width = bbox[2] - bbox[0]
        header_height = bbox[3] - bbox[1] +1000
        gradient_header = self.create_gradient_text_image(header_text, font_header, 
                                                          (255, 255, 255), (83, 234, 205), 
                                                          header_width, header_height)
        
        # Position header on background
        header_x = (background.width - header_width) // 2
        header_y = 100  # Arbitrary top padding
        background.paste(gradient_header, (header_x, header_y), gradient_header)

        # Position logo at the bottom right
        logo.thumbnail((200, 200))  # Adjust size
        background.paste(logo, (background.width - logo.width - 20, background.height - logo.height - 20), logo)

        # Draw list items
        draw = ImageDraw.Draw(background)
        y_position = header_y + header_height + 100  # Space below header

        for item in list_items:
            draw.text((header_x, y_position), item, font=font_items, fill=(255, 255, 255))
            y_position += font_items.getbbox(item)[1] + 50  # Line spacing

        return background

# The update should now apply the gradient directly onto the text without leaving a white overlay.


In [44]:
creator = ListSlideCreator()
slide = creator.create_slide("Header", ["Item 1", "Item 2", "Item 3"])
slide.show()